In [ ]:
import os
import json
import cv2
import numpy as np
from tqdm import tqdm
from collections import Counter
import pickle

IGNORE_LABEL = 65535  # ADE20K ignore label, it is a left class like a background
TARGET_SIZE = (384, 384)  # the final resolution of images

In [ ]:
# function to resize images
def resize_images(source_dir, output_dir, target_size=(384, 384)):
   
    os.makedirs(output_dir, exist_ok=True)
    
    image_files = [f for f in os.listdir(source_dir) 
                   if f.endswith(('.jpg', '.png', '.jpeg'))]
    
    for img_file in tqdm(image_files, desc="Resizing images"):
        img_path = os.path.join(source_dir, img_file)
        img = cv2.imread(img_path)
        
        # Resize
        img_resized = cv2.resize(img, (target_size[1], target_size[0]))
        
        # save
        output_path = os.path.join(output_dir, img_file)
        cv2.imwrite(output_path, img_resized)
    
#saved in {output_dir}

In [ ]:
# function for creation masks for every class 
def create_full_masks(annotation_dir, output_mask_dir, target_size=(384, 384)):

    os.makedirs(output_mask_dir, exist_ok=True)
    
    json_files = [f for f in os.listdir(annotation_dir) if f.endswith(".json")]
    
    for json_file in tqdm(json_files, desc="Creating full masks"):
        json_path = os.path.join(annotation_dir, json_file)
        
        with open(json_path, "r") as f:
            data = json.load(f)
        
        height = data["size"]["height"]
        width = data["size"]["width"]
        
        # init mask with ignore(background) label
        mask = np.full((height, width), IGNORE_LABEL, dtype=np.uint16)
        
        # draw every objjects
        objects = data.get("objects", [])
        objects_sorted = sorted(objects, key=lambda x: x.get("area", 0), reverse=True)
        
        for obj in objects_sorted:
            class_id = obj["classId"]
            
            # exterior polygon
            ext_pts = obj["points"].get("exterior", [])
            if len(ext_pts) >= 3:
                poly = np.array(ext_pts, dtype=np.int32).reshape((-1, 1, 2))
                cv2.fillPoly(mask, [poly], class_id)
            
            # interior polygons 
            for int_pts in obj["points"].get("interior", []):
                if len(int_pts) >= 3:
                    poly = np.array(int_pts, dtype=np.int32).reshape((-1, 1, 2))
                    cv2.fillPoly(mask, [poly], IGNORE_LABEL)
        
        # resize to 384 x 384
        mask_resized = cv2.resize(
            mask, 
            (target_size[1], target_size[0]), 
            interpolation=cv2.INTER_NEAREST
        )
        
        # save as png)
        mask_name = json_file.replace(".json", ".png")
        cv2.imwrite(os.path.join(output_mask_dir, mask_name), mask_resized)
    
# saved to {output_mask_dir}

In [ ]:
# pixel distribution analysis after resising masks 
def analyze_pixel_distribution(mask_dir, ignore_label=65535):

    pixel_counter = Counter()
    mask_files = sorted([f for f in os.listdir(mask_dir) if f.endswith('.png')])
    
    for mask_file in tqdm(mask_files, desc="Counting pixels"):
        mask_path = os.path.join(mask_dir, mask_file)
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        
        if mask is None:
            continue
        
        # counting pixels for every class
        unique, counts = np.unique(mask, return_counts=True)
        
        for class_val, count in zip(unique, counts):
            if class_val != ignore_label:
                pixel_counter[class_val] += count
    
    return pixel_counter

In [ ]:
# Getting top N(80%) classes
def get_top_percent_classes(pixel_counter, annotation_dir, target_percent=80):
    
    pixel_counter = {k: v for k, v in pixel_counter.items() 
                     if k != IGNORE_LABEL}
    
    if not pixel_counter:
        print("⚠️ Pixel counter пуст!")
        return [], {}
    
    # collecting classId to classTitle
    class_titles = {}
    json_files = [f for f in os.listdir(annotation_dir) if f.endswith(".json")]
    
    for json_file in tqdm(json_files[:100], desc="Building class titles"):
        json_path = os.path.join(annotation_dir, json_file)
        with open(json_path, "r") as f:
            data = json.load(f)
        
        for obj in data.get("objects", []):
            class_id = obj["classId"]
            class_title = obj.get("classTitle", f"class_{class_id}")
            if class_id != IGNORE_LABEL:
                class_titles[class_id] = class_title
    
    # sorting pixel by pixel
    total_pixels = sum(pixel_counter.values())
    sorted_classes = sorted(pixel_counter.items(), key=lambda x: x[1], reverse=True)
    
    cumulative_pixels = 0
    top_classes = []
    
    for class_id, px_count in sorted_classes:
        top_classes.append(class_id)
        cumulative_pixels += px_count
        if cumulative_pixels / total_pixels >= target_percent / 100:
            break
    
#stats
    print(f"Number of classes in resized datast: {len(pixel_counter)}")
    print(f"Classes {target_percent}% for pixels: {len(top_classes)}")
    print(f"Coverage: {cumulative_pixels:,} / {total_pixels:,} of pixels ({cumulative_pixels/total_pixels*100:.2f}%)")
    
    print(f"\n📋 top-{len(top_classes)} clasees:")
    for i, class_id in enumerate(top_classes, 1):
        title = class_titles.get(class_id, f"class_{class_id}")
        px_count = pixel_counter[class_id]
        pct = px_count / total_pixels * 100
        print(f"{i:3d}. {class_id:6d} : {title:30s} {px_count:12,} ({pct:6.2f}%)")
    
    return top_classes, class_titles

In [ ]:
# creating final masks with selected classes 
def create_filtered_masks(full_mask_dir, output_mask_dir, selected_classes, 
                         class_id_to_idx, ignore_label=65535):
    
    os.makedirs(output_mask_dir, exist_ok=True)
    
    mask_files = sorted([f for f in os.listdir(full_mask_dir) if f.endswith('.png')])
        
    stats = {'total': 0, 'empty': 0, 'with_objects': 0}
    
    for mask_file in tqdm(mask_files, desc="Creating filtered masks"):
        mask_path = os.path.join(full_mask_dir, mask_file)
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        
        # create new mask with new indexes
        mask_remapped = np.full_like(mask, 255, dtype=np.uint8)
        
        objects_found = False
        for class_id, new_idx in class_id_to_idx.items():
            mask_remapped[mask == class_id] = new_idx
            if np.any(mask == class_id):
                objects_found = True
        
        stats['total'] += 1
        if objects_found:
            stats['with_objects'] += 1
        else:
            stats['empty'] += 1
        
        # save
        output_path = os.path.join(output_mask_dir, mask_file)
        cv2.imwrite(output_path, mask_remapped)


In [ ]:
# the main pipeline with calling all functions above
DATASET_ROOT = "./dataset2" #place where original dataset is located
TARGET_PERCENT = 80  # getting 80% of all classes


# directories
train_img_source = os.path.join(DATASET_ROOT, "training", "img")
train_ann_source = os.path.join(DATASET_ROOT, "training", "ann")
val_img_source = os.path.join(DATASET_ROOT, "validation", "img")
val_ann_source = os.path.join(DATASET_ROOT, "validation", "ann")

# Resized folders
resized_root = os.path.join(DATASET_ROOT, "resized")
train_img_resized = os.path.join(resized_root, "train", "images")
train_mask_full = os.path.join(resized_root, "train", "masks_full")
val_img_resized = os.path.join(resized_root, "val", "images")
val_mask_full = os.path.join(resized_root, "val", "masks_full")

# final folders
final_root = os.path.join(DATASET_ROOT, "processed_final")
train_mask_final = os.path.join(final_root, "train", "masks")
val_mask_final = os.path.join(final_root, "val", "masks")


In [ ]:
# images resizing


print("\nTrain images...")
resize_images(train_img_source, train_img_resized, TARGET_SIZE)

print("\nValidation images...")
resize_images(val_img_source, val_img_resized, TARGET_SIZE)

In [ ]:
# creating masks with all classes
#Train masks
create_full_masks(train_ann_source, train_mask_full, TARGET_SIZE)

#Validation masks
create_full_masks(val_ann_source, val_mask_full, TARGET_SIZE)

In [ ]:
pixel_counter = analyze_pixel_distribution(train_mask_full, ignore_label=IGNORE_LABEL)

In [ ]:
#getting top 80% of classes
selected_classes, class_titles = get_top_percent_classes(
    pixel_counter, 
    train_ann_source, 
    target_percent=TARGET_PERCENT
)

In [ ]:

#Creating mappings and sorting it
selected_classes_sorted = sorted(selected_classes)
class_id_to_idx = {class_id: idx for idx, class_id in enumerate(selected_classes_sorted)}
idx_to_class_id = {idx: class_id for class_id, idx in class_id_to_idx.items()}


In [ ]:
#Train masks
create_filtered_masks(train_mask_full, train_mask_final, selected_classes, 
                     class_id_to_idx, IGNORE_LABEL)

#Validation masks
create_filtered_masks(val_mask_full, val_mask_final, selected_classes, 
                     class_id_to_idx, IGNORE_LABEL)

In [ ]:
# saving mappings
mappings = {
    'selected_classes': selected_classes_sorted,
    'class_titles': class_titles,
    'class_id_to_idx': class_id_to_idx,
    'idx_to_class_id': idx_to_class_id,
    'num_classes': len(selected_classes_sorted),
    'target_size': TARGET_SIZE,
    'target_percent': TARGET_PERCENT
}
#sacing mappings
mappings_path = os.path.join(DATASET_ROOT, "class_mappings_final.pkl")
with open(mappings_path, "wb") as f:
    pickle.dump(mappings, f)


In [ ]:

# images: resized/train(val)/images
#m asks:  processed_final/train(val)/masks

#Train: ./dataset2\resized\train\images
#     Val:   ./dataset2\resized\val\images

#   final msaks (with 0-13 mapping):
#     Train: ./dataset2\processed_final\train\masks
#     Val:   ./dataset2\processed_final\val\masks